# 基于平均感知器的分词器

## 原理

&emsp;&emsp;我们知道对于一句话进行分词，我们可以通过给每个字进行标注状态（B：分词词首；M：分词词中；E：分词词尾；S：单个词分词）来分词，然后去计算出每个字到底是什么状态，从而完成分词。

&emsp;&emsp;那使用感知器，无非也就是用感知器来预测出每个字的4种状态的概率分别是多少，然后利用维特比算法找出整个句子最大概率的状态路径，从而完成分词。和hmm分词差不多，只是将hmm预测隐状态概率换成了使用感知器的多分类来预测隐状态概率。

In [ ]:
from collections import defaultdict
import pickle
import random


class AveragedPerceptron(object):

    '''An averaged perceptron, as implemented by Matthew Honnibal.
    See more implementation details here:
        http://honnibal.wordpress.com/2013/09/11/a-good-part-of-speechpos-tagger-in-about-200-lines-of-python/
    '''

    def __init__(self):
        # Each feature gets its own weight vector, so weights is a dict-of-dicts
        self.weights = {}
        self.classes = set()
        # The accumulated values, for the averaging. These will be keyed by
        # feature/clas tuples
        self._totals = defaultdict(int)
        # The last time the feature was changed, for the averaging. Also
        # keyed by feature/clas tuples
        # (tstamps is short for timestamps)
        self._tstamps = defaultdict(int)
        # Number of instances seen
        self.i = 0

    def predict(self, features):
        '''Dot-product the features and current weights and return the best label.'''
        scores = defaultdict(float)
        for feat, value in features.items():
            if feat not in self.weights or value == 0:
                continue
            weights = self.weights[feat]
            for label, weight in weights.items():
                scores[label] += value * weight
        # Do a secondary alphabetic sort, for stability
        return max(self.classes, key=lambda label: (scores[label], label))

    def update(self, truth, guess, features):
        '''Update the feature weights.'''
        def upd_feat(c, f, w, v):
            param = (f, c)
            self._totals[param] += (self.i - self._tstamps[param]) * w
            self._tstamps[param] = self.i
            self.weights[f][c] = w + v

        self.i += 1
        if truth == guess:
            return None
        for f in features:
            weights = self.weights.setdefault(f, {})
            upd_feat(truth, f, weights.get(truth, 0.0), 1.0)
            upd_feat(guess, f, weights.get(guess, 0.0), -1.0)
        return None

    def average_weights(self):
        '''Average weights from all iterations.'''
        for feat, weights in self.weights.items():
            new_feat_weights = {}
            for clas, weight in weights.items():
                param = (feat, clas)
                total = self._totals[param]
                total += (self.i - self._tstamps[param]) * weight
                averaged = round(total / float(self.i), 3)
                if averaged:
                    new_feat_weights[clas] = averaged
            self.weights[feat] = new_feat_weights
        return None

    def save(self, path):
        '''Save the pickled model weights.'''
        return pickle.dump(dict(self.weights), open(path, 'w'))

    def load(self, path):
        '''Load the pickled model weights.'''
        self.weights = pickle.load(open(path))
        return None


def train(nr_iter, examples):
    '''Return an averaged perceptron model trained on ``examples`` for
    ``nr_iter`` iterations.
    '''
    model = AveragedPerceptron()
    for i in range(nr_iter):
        random.shuffle(examples)
        for features, class_ in examples:
            scores = model.predict(features)
            guess, score = max(scores.items(), key=lambda i: i[1])
            if guess != class_:
                model.update(class_, guess, features)
    model.average_weights()
    return model

## 数据处理
1、对每个句子中的每个字，取7种特征【unigrams+bigrams】

> $C_{i-1}$ #前一个词 \
> $C_i$ #当前词 \
> $C_{i+1}$ #后一个词 \
> $C_{i-2}/C_{i-1}$ #前两个词与前一个词 \
> $C_{i-1}/C_i$ #前一个词与当前词 \
> $C_i/C_{i+1}$  #当前词与后一个词 \
> $C_{i+1}/C_{i+2}$  #后一个词与后两个词


In [ ]:
data_f = open('./data/RenMinData.txt', r, encoding='utf-8')
for line in data_f.readlinese:
    